# LDF

The output from the stand-alone LDF configuration is in the link below


Note that LDF is currently run by users. Temporarily, we suggest users walk through the following process:
1) Install LDF and activate cupid-analysis
2) Use the `CUPiD/helper_scripts/generate_ldf_config_file.py` script to generate a LDF config file based on a CUPiD configuration file.
   * `cd CUPiD/examples/external_diag_packages`
   * `../../helper_scripts/generate_ldf_config_file.py --cupid-config-loc . --ldf-template ../../externals/LDF/config_clm_unstructured_plots.yaml --out-file LDF_config.yaml`
2) Run LDF with the newly created configuration file.
   * `../../externals/LDF/run_adf_diag LDF_config.yaml`

In [ ]:
import os

from IPython.core.display import HTML, Image
from IPython.display import display
import pandas as pd

In [ ]:
ldf_root = "."
case_name = None
base_case_name = None
start_date = ""
end_date = ""
base_start_date = None
base_end_date = None
key_plots = None
# ldf_root will be external_diag_packages/computed_notebooks/LDF/

In [ ]:
# Want some base case parameter defaults to equal control case values
if base_case_name is not None:
    if base_start_date is None:
        base_start_date = start_date

    if base_end_date is None:
        base_end_date = end_date

In [ ]:
# convert start-date and end-date to year range
case_year_range = [int(start_date.split("-")[0]), int(end_date.split("-")[0]) - 1]

base_case_year_range = [
    int(base_start_date.split("-")[0]),
    int(base_end_date.split("-")[0]) - 1,
]

In [ ]:
ldf_comparison_name = f"{case_name}_{case_year_range[0]}_{case_year_range[1]}_vs_{base_case_name}_{base_case_year_range[0]}_{base_case_year_range[1]}"
ldf_comparison_name_v1 = f"{case_name}_{case_year_range[0]}_{str(int(case_year_range[1])-1)}_vs_{base_case_name}_{base_case_year_range[0]}_{base_case_year_range[1]}"
ldf_comparison_name_v2 = f"{case_name}_{case_year_range[0]}_{str(int(case_year_range[1])-1)}_vs_{base_case_name}_{base_case_year_range[0]}_{str(int(base_case_year_range[1])-1)}"
ldf_comparison_name_v3 = f"{case_name}_{case_year_range[0]}_{case_year_range[1]}_vs_{base_case_name}_{base_case_year_range[0]}_{str(int(base_case_year_range[1])-1)}"

ldf_root = os.path.join(ldf_root, ldf_comparison_name)
ldf_root_v1 = os.path.join(ldf_root, ldf_comparison_name_v1)
ldf_root_v2 = os.path.join(ldf_root, ldf_comparison_name_v2)
ldf_root_v3 = os.path.join(ldf_root, ldf_comparison_name_v3)

if os.path.exists(ldf_root):
    display(
        HTML(
            f'<a href="../LDF/{ldf_comparison_name}/website/index.html" target="_blank" style="font-size: 30px">Full LDF output</a>'
        )
    )

# ADF/LDF often have one or both end years off by one due to data availability. Check for those cases here, and check ldf_root_v1 first in case of base case being none.
elif os.path.exists(ldf_root_v1):
    ldf_root = ldf_root_v1  # this line should fix missing key plots if a different directory is found
    display(
        HTML(
            f'<a href="./LDF/{ldf_comparison_name_v1}/website/index.html" target="_blank" style="font-size: 30px">Full LDF output</a>'
        )
    )
elif os.path.exists(ldf_root_v2):
    ldf_root = ldf_root_v2
    display(
        HTML(
            f'<a href="./LDF/{ldf_comparison_name_v2}/website/index.html" target="_blank" style="font-size: 30px">Full LDF output</a>'
        )
    )
elif os.path.exists(ldf_root_v3):
    ldf_root = ldf_root_v3
    display(
        HTML(
            f'<a href="./LDF/{ldf_comparison_name_v2}/website/index.html" target="_blank" style="font-size: 30px">Full LDF output</a>'
        )
    )
else:
    print("No LDF output found for the specified case and date range.") 

## Key Metrics from LDF

Some important things to look at from LDF include a comparison table and a few maps:

In [ ]:
comparison_table = os.path.join(ldf_root, "amwg_table_comp.csv")
if os.path.isfile(comparison_table):
    table = pd.read_csv(comparison_table)
    display(HTML(table.to_html(index=False, float_format="{:6g}".format)))

In [ ]:
for path_to_key_plot in key_plots:
    full_path = os.path.join(ldf_root, path_to_key_plot)
    if os.path.isfile(full_path):
        display(Image(full_path))